# 0. import

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# pytorch 相關
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data import random_split
from torchvision import models
# 其他
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from copy import copy
import os
import wandb
from wandb.keras import WandbCallback
import sklearn.metrics as metrics
import seaborn as sns
import pandas as pd

In [ ]:
# check GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU state:', device)

# 1. 設置超參數

In [ ]:
lr = 0.0001
batch_size = 128
epochs = 50
model_path = './m.pth'
train_path = '/kaggle/input/hw2-dataset/HW2_dataset/HW2_dataset/training'

# 2. Wandb

In [ ]:
# 紀錄數據
wandb.login(key = "bf1bc673d9b5cb8bf02f1937561fb29fcb06a207")
wandb.init(
    project = 'vgg16', 
    name = 'batch_size: %d' % batch_size ,
    job_type = "training" , 
    )
config = wandb.config
config.batch_size = batch_size

# 3. 數據預處理

In [ ]:
# Normalize 和 totensor
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 切割train_validation & load_data

In [ ]:
# 因需切割 train data 為 train 與 validation，所以需先讀入data並切割
# 讀資料
import glob
# 找出 train_path 底下所有檔案
train_list = glob.glob(os.path.join(train_path , '**/*.*') , recursive = True)

# 切割
from sklearn.model_selection import train_test_split
train_list , val_list = train_test_split(train_list , test_size = 0.2 , random_state = 2)

In [ ]:
# 直接定義 class list 順序
cls_list = ['Jackson_Pollock', 'Alfred_Sisley', 'Jan_van_Eyck', 'Kazimir_Malevich', 'Sandro_Botticelli', 'Rembrandt', 'Vincent_van_Gogh', 'Raphael', 'Frida_Kahlo', 'Georges_Seurat', 'Edouard_Manet', 'Michelangelo', 'Salvador_Dali', 'Giotto_di_Bondone', 'Diego_Rivera', 'Peter_Paul_Rubens', 'William_Turner', 'Leonardo_da_Vinci', 'Piet_Mondrian', 'Vasiliy_Kandinskiy', 'Titian', 'Paul_Gauguin', 'Francisco_Goya', 'Edgar_Degas', 'Pablo_Picasso', 'Henri_Rousseau', 'Paul_Cezanne', 'Andy_Warhol', 'Mikhail_Vrubel', 'Diego_Velazquez', 'Hieronymus_Bosch', 'Eugene_Delacroix', 'Andrei_Rublev', 'Gustav_Klimt', 'Henri_Matisse', 'Gustave_Courbet', 'Caravaggio', 'Joan_Miro', 'Pierre-Auguste_Renoir', 'Rene_Magritte', 'Camille_Pissarro', 'Henri_de_Toulouse-Lautrec', 'El_Greco', 'Claude_Monet', 'Marc_Chagall', 'Paul_Klee', 'Edvard_Munch', 'Amedeo_Modigliani', 'Pieter_Bruegel']

# 4. 載入 Dataset

In [ ]:
class HW2(Dataset):
  def __init__(self, file_list , transform=None):
    self.file_list = file_list
    self.transform = transform
    # return 該路徑下之文件和文件夾列表
    # self.classes = os.listdir(train_path)
    self.classes = cls_list
    # dict : class_name對應index
    self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
    self.images = self.load_images()

  def get_classes(self):
    return self.classes

  def load_images(self):
    images = []
    for img_path in self.file_list:
      # 從 path 中切割出 class 名
      cls_name = img_path.split("/")[7]
      images.append((img_path, self.class_to_idx[cls_name]))
    return images

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, idx):
    img_path, label = self.images[idx]
    # 固定 image 的大小
    img = Image.open(img_path).resize([200 , 200]).convert('RGB')
    # 若有定義transform
    if self.transform:
        img = self.transform(img)
    return img, label

In [ ]:
train_dataset = HW2(train_list , train_transform)
val_dataset = HW2(val_list , val_transform)

In [ ]:
classes = train_dataset.get_classes()
print(classes)

# 5. 載入 Dataloader

In [ ]:
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=2)
val_loader = DataLoader(dataset=val_dataset,batch_size=batch_size,shuffle=False,num_workers=2)

# 6. 定義模型

In [ ]:
model = models.vgg19(weights='DEFAULT')

In [ ]:
num_fcin = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_fcin, len(classes))
model.to(device)

# 7. 定義 Cost function

In [ ]:
# 計算自定義 weight (先計算每個 class 數量) - 實作 penalized loss
class_num = []
for c in cls_list :
    p = os.path.join(train_path , c)
    num = len(os.listdir(p))
    class_num.append(num)
weights = []
for i in range(len(class_num)) : 
    weights.append(sum(class_num) / class_num[i])
class_weights = torch.FloatTensor(weights).to(device)

# 套入 loss 的 function 來改變計算權重
loss = nn.CrossEntropyLoss(weight = class_weights)
optimizer = optim.SGD(model.parameters(), lr, momentum=0.9,weight_decay=5e-4)
# optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# 8. 開始訓練

In [ ]:
# best model accurancy
best_acc = 0.0
train_loss = []
train_acc = []
val_loss = []
val_acc = []
macro_pre = []
macro_recall = []
micro_pre = []
micro_recall = []

# 實作 learning rate 隨著 epoch 改變
def adjust_learning_rate(optimizer, epoch):
  if epoch <= 20:
    lr = 0.001
  elif epoch <= 40:
    lr = 0.001
  else:
    lr = 0.0001
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr

for epoch in range(epochs):
  # train
  train_epoch_loss = 0.0

  train_class_correct = list(0. for i in range(len(classes)))
  train_class_total = list(0. for i in range(len(classes)))

  # 每個 epoch 前都去看看要不要調 learning rate
  adjust_learning_rate(optimizer, epoch)
  
  # Evaluation Metrics 會用到
  total_label = np.array([])
  total_predicted = np.array([])

  model.train()
  for i, data in enumerate(train_loader, 0):
    # enumerate 回傳的是 (下標 , data)

    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)

    # Compute Loss & Update Weight
    batch_loss = loss(outputs, labels)
    #　back propagation
    batch_loss.backward()
    # 更新
    optimizer.step()

    train_epoch_loss += batch_loss.item()

    # Compute train_class_correct of each batch
    _, predicted = torch.max(outputs, 1)
    # predicted 為預測出來之 label , 1 為 dim = 1 列出每行的最大值
    batch_correct = (predicted == labels)
    # 回傳一個 tensor 用於計算正確幾個
    for j in range(len(labels)):
      label = labels[j]
      train_class_correct[label] += batch_correct[j].item()
      train_class_total[label] += 1

  # Compute Loss & Acc
  train_epoch_loss = train_epoch_loss / len(train_loader)
  train_epoch_accuracy = sum(train_class_correct) / sum(train_class_total) * 100

  train_loss.append(train_epoch_loss)
  train_acc.append(train_epoch_accuracy)

  print()
  print('[Epoch:%2d]' % (epoch + 1))
  print('Train Accuracy of All : %.3f %%' % (train_epoch_accuracy))
  print('Train Loss of All : %.3f ' % (train_epoch_loss))
  print("----------------------------------------")

    
  # Validation class correct & class total --------------------------------------------------------------
  model.eval()
  val_epoch_loss = 0.0
  val_class_correct = list(0. for i in range(len(classes)))
  val_class_total = list(0. for i in range(len(classes)))

  # Validation every epoch
  with torch.no_grad():
    for data in val_loader :
      images, labels = data
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      # 1. Compute val_batch_loss
      batch_loss = loss(outputs, labels)
      val_epoch_loss += batch_loss.item()

      # 2. Compute val_class_correct of each batch
      _, predicted = torch.max(outputs, 1)
      batch_correct = (predicted == labels)
      for j in range(len(labels)):
        label = labels[j]
        val_class_correct[label] += batch_correct[j].item()
        val_class_total[label] += 1
      
      # 整理所有結果
      total_label = np.append(total_label , labels.cpu().numpy())
      total_predicted = np.append(total_predicted , predicted.cpu().numpy())

  # Evaluation Metrics
  val_epoch_accuracy = sum(val_class_correct) / sum(val_class_total) * 100
  val_epoch_loss = val_epoch_loss / len(val_loader)
  mac_epoch_pre = metrics.precision_score(total_label, total_predicted, average = 'macro') * 100
  mac_epoch_recall = metrics.recall_score(total_label, total_predicted, average = 'macro') * 100
  mic_epoch_pre = metrics.precision_score(total_label, total_predicted, average = 'micro') * 100
  mic_epoch_recall = metrics.recall_score(total_label, total_predicted, average = 'micro') * 100

  val_loss.append(val_epoch_loss)
  val_acc.append(val_epoch_accuracy)
  macro_pre.append(mac_epoch_pre)
  macro_recall.append(mac_epoch_recall)
  micro_pre.append(mic_epoch_pre)
  micro_recall.append(mic_epoch_recall)
    
  wandb.log( {"train_accuracy" : train_epoch_accuracy , "epoch" : epoch} )
  wandb.log( {"train_loss" : train_epoch_loss , "epoch" : epoch} )
  wandb.log( {"validation_accuracy" : val_epoch_accuracy , "epoch" : epoch} )
  wandb.log( {"validation_loss" : val_epoch_loss , "epoch" : epoch} )
  wandb.log( {"macro_precision" : mac_epoch_pre , "epoch" : epoch} )
  wandb.log( {"macro_recall" : mac_epoch_recall , "epoch" : epoch} )
  wandb.log( {"micro_precision" : mic_epoch_pre , "epoch" : epoch} )
  wandb.log( {"micro_recall" : mic_epoch_recall , "epoch" : epoch} )
  
  print('Validation Accuracy of All : %.3f %%' % (val_epoch_accuracy))
  print('Validation Loss of All : %.3f ' % (val_epoch_loss))
  print('Validation Macro-averaged Precision of All : %.3f %%' % (mac_epoch_pre))
  print('Validation Macro-averaged Recall of All : %.3f %%' % (mac_epoch_recall))
  print('Validation Micro-averaged Precision of All : %.3f %%' % (mic_epoch_pre))
  print('Validation Micro-averaged Recall of All : %.3f %%' % (mic_epoch_recall))
  print("----------------------------------------")

  # Save best model
  if val_epoch_accuracy > best_acc:
    best_acc = val_epoch_accuracy
    torch.save(model.state_dict() , model_path)
print('Finished Training')
wandb.finish()

# 9. 測試

In [ ]:
# 載入訓練好的 model
m = models.vgg19()
num_fcin = m.classifier[6].in_features
m.classifier[6] = nn.Linear(num_fcin, len(classes))
w = torch.load("/kaggle/input/mmmmmm/m (1).pth")
m.load_state_dict(w)
m = m.cuda()
m.eval()

class_to_index = {cls_name: i for i, cls_name in enumerate(cls_list)}
test_path = '/kaggle/input/hw2-dataset/HW2_dataset/HW2_dataset/testing'
test = pd.read_csv("/kaggle/input/test-csv/Homework2.csv")
pre = []

with torch.no_grad():
  for data in test["image"] :
      # 找到該 image
      img_path = os.path.join(test_path , data)
      img = Image.open(img_path).resize([200 , 200]).convert('RGB')
      img = test_transform(img)
      images = img
      images = images.to(device)
      # reshape 成 Model 要的
      images = torch.reshape(images , (1 , 3 , 200 , 200))
      outputs = m(images)

      _, predicted = torch.max(outputs, 1)
      p = predicted.data.cpu().numpy()[0]
      # 記錄起來該結果
      pre.append(p)

# 寫入 dataframe 中
for i in range(len(pre)) :
    test["Painter"][i] = cls_list[pre[i]]

# 匯出
test.to_csv("result.csv")
# 再手動將第一 column 刪除